In [1]:
import tensorflow as tf
import requests
import pandas as pd
import numpy as np

2022-11-16 04:02:26.536419: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-16 04:02:26.539212: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaParaTestePorGrupo?idGrupo=2"

r = requests.get(url)
print(url)
j = r.json()
dfTreino = pd.DataFrame.from_dict(j)
dfTreino.head()

http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaParaTestePorGrupo?idGrupo=2


,id,grupoAcaoId,periodoTreinoRedeId,tipoExemploTreinoId,regraProjecaoId,redeNeuralId,ativoPrevisao,limiteParaEntrada,qtdeLucroTeste,qtdePrejuizoTeste,...,dataPrevisaoTeste,mediaValorTeste,desvioValorTeste,qtdeTradeTeste,pontuacaoExecucao,qtdeLucroExecucao,qtdePrejuizoExecucao,tipoCompraVenda,diaNumInicioTeste,diaNumFinalTeste
0,405,12,2,None,70,9,0,0.5,None,None,...,None,None,None,None,None,None,None,V,20220501,20221031
1,423,12,2,None,70,7,0,0.5,None,None,...,None,None,None,None,None,None,None,V,20220501,20221031
2,465,12,2,None,70,8,0,0.5,None,None,...,None,None,None,None,None,None,None,V,20220501,20221031


In [3]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [4]:

def incluiEntrada(treino):
    urlEntrada = "http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoPorDiaB3?diaNum=" + str(diaCorrente) +"&idRegra=" + str(treino['regraProjecaoId']) + "&idGrupo=" + str(treino['grupoAcaoId'])
    #print(urlEntrada)
    r = requests.get(urlEntrada)
    j = r.json()
    dfEntradas = pd.DataFrame.from_dict(j)
    #print(dfEntradas.head())
    return dfEntradas

In [5]:
def trataRede():
    treino = treinoCorrente
    #print('ID:' , treino['id'])
    #print(nome_arquivo)
    #modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(treino['id'])
    dfDados['tipoCompraVenda'] = treino['tipoCompraVenda']
    dfEntradas = incluiEntrada(treino)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    #print('ticker previsão:' , len(dfPrevisao))
    json = dfPrevisao.to_dict(orient='list')
    urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoTestes/recebePrevisaoTreinoTeste'
    #print(json)
    response = requests.post(urlPrevisao, data=json)
    print(response)

In [6]:
def trataDiaPregao(dia):
    print('Tratando Dia: ' , dia.diaNum)
    global diaCorrente
    diaCorrente = dia.diaNum
    urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDiaNum?diaNum={}".format(dia.diaNum)
    #print(urlExemplos)
    r = requests.get(urlExemplos)
    j = r.json()
    global dfDados
    dfDados = pd.DataFrame.from_dict(j)
    #print(dfDados.head())
    dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
    global X
    X = np.stack(dfTratadoX.values)
    X.shape
    dfDados.drop('campoX', inplace=True, axis=1)
    trataRede()

In [7]:
def trataTreinoRede(treino):
    print('Tratando Rede: ' , treino.id)
    global treinoCorrente
    treinoCorrente = treino
    nome_arquivo = '/root/executa-rede/redes/treino_rede' + str(treino['id']) + '.h5'
    global modelo
    modelo = tf.keras.models.load_model(nome_arquivo)

    print('TreinoCorrente: ' , treinoCorrente)
    urlDias = "http://vps-40d69db1.vps.ovh.ca:23004/api/DiaPregaos/obtemPeriodoPrevisaoTeste?idPeriodoTreinoRede={}".format(treino.periodoTreinoRedeId)
    r = requests.get(urlDias)
    j = r.json()
    dfDias = pd.DataFrame.from_dict(j)
    print(dfDias.head())
    dfDias.apply(trataDiaPregao, axis='columns')

    urlListaTeste = 'http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/executouDadoTeste'
    response = requests.post(urlListaTeste, data={'id' : treino.id})
    print('Finalizacao de Treino')
    print(response)

In [ ]:
dfTreino.apply(trataTreinoRede, axis='columns')